In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
%pylab inline

ModuleNotFoundError: No module named 'seaborn'

# 1. Загрузка данных

В качестве исходных данных используем массив о подлинных и истинных купюрах (https://archive.ics.uci.edu/ml/datasets/banknote+authentication).
Данные были извлечены из изображений, которые были взяты из подлинных и поддельных образцов, похожих на банкноты. Для оцифровки использовалась промышленная камера, обычно используемая для проверки печати. Окончательные изображения имеют 400x 400 пикселей. Исходные признаки были получены с помощью преобразования Вейвлета.

$x^2$

In [ ]:
data = pd.read_csv('data_banknote_authentication.txt', sep=',', header=None)

Посмотрим несколько записей из массива.

In [ ]:
data.head(10)

Переименуем названия признаков в интерпретируемые.
* variance - дисперсия
* skewness - ассиметрия
* kurtosis - эксцесс
* entropy - энтропия изображения
* class - выходной класс

In [ ]:
data.columns = ['variance', 'skewness', 'kurtosis', 'entropy', 'class']

In [ ]:
data.head()

# 2. Предварительный анализ

Количество элементво в выборке.

In [ ]:
data.shape

Итого: 1372 записи (немного), 5 признаков.

### Изучим исходные признаки.

In [ ]:
features = ['variance', 'skewness', 'kurtosis', 'entropy']
data[features].info()

Сразу видно, что в признаке variance имеются пропуски.

In [ ]:
data[features].describe()

In [ ]:
data['entropy'].value_counts()

In [ ]:
data[data['entropy'] != '?']['entropy']

In [ ]:
d = {'?': 10}
data['entropy'] = data['entropy'].replace(d).astype('float')

Построим гистограммы для исходных признаков.

In [ ]:
data[features].hist(bins=15, figsize=(15,7))

### Проанализируем целевую переменную.

In [ ]:
data['class'] = data['class'].astype('bool')
data['class'].describe()

In [ ]:
data['class'].hist(bins=2)
print data['class'].value_counts()

Классы немного несбалансированы, но не страшно.

### Корреляционный анализ

In [ ]:
print data.corr()
sns.heatmap(data.corr())

Поскольку считать корреляцию между категориальной переменной и вещественной некорректно, построим диаграмму рассеяния для дисперсии и целевой переменной.

In [ ]:
sns.regplot(x="variance", y="class", data=data)

In [ ]:
data.plot.scatter(x="variance", y="class")

Видно, что при больших значениях дисперсии вероятность получить фальшивую купюру больше.

# 3. Подготовка данных

Из процедур подготовки данных необходимо только заполнить пропущенные значения для признака variance. Поскольку минимальное значение -7, а максимальное 6.8, заполним пропущенные значения -15.

In [ ]:
data['variance'].fillna(-15, inplace=True)

Разделим исходную выборку на X и y.

In [ ]:
X = data[features]
y = data['class']

# 4. Обучение и тестирование моделей

Итак, начнем обучать и тестировать модели.

Для начала зафиксируем random_state.

In [ ]:
random_state = 15

### Критерии расщепления

Проведем сравнение ДР с параметрами по умолчанию, но с разными критериями расщепления.

In [ ]:
criteria = ['gini', 'entropy']

for cr in criteria:
    # ToDo
    clf = DecisionTreeClassifier(...)
    scores = cross_val_score(...)
    print 'Criterion: {}, sccuracy score: {}'.format(cr, scores.mean())

Видим, что модели с параметрами по умолчанию уже дали прекрасные результаты. Посмотрим, можно ли что-то улучшить.

### Количество признаков (max_features)

In [ ]:
max_features = range(2, 5)

for mf in [2,3,4]:
    # ToDo
    clf = DecisionTreeClassifier(...)
    scores = cross_val_score(...)
    print 'Criterion: {}, sccuracy score: {}'.format(cr, scores.mean())

Оптимальное количество = 3.

### Глубина дерева (max_depth)

In [ ]:
max_depth = range(2, 20)
scoring = []

for md in max_depth:
    # ToDo
    clf = DecisionTreeClassifier(...)
    scores = cross_val_score(...)
    print 'Criterion: {}, sccuracy score: {}'.format(cr, scores.mean())
    
plt.plot(max_depth, scoring)
plt.xlabel('Depth')
plt.ylabel('Accuracy')

Итак, оптимальная глубина = 8.

# 5. Визуализация ДР

Выведем в pdf дерево решений с оптимальными параметрами и глубиной 3.

In [ ]:
from sklearn.tree import export_graphviz

for depth in [3,8]:
    clf = DecisionTreeClassifier(criterion='entropy', max_features=3, max_depth=depth, random_state=random_state)
    clf.fit(X, y)
    export_graphviz(clf, out_file='tree{}.dot'.format(depth))

Теперь из dot файла можно сгенерировать png.

In [ ]:
!dot -Tpng tree3.dot -o tree3.png
!dot -Tpng tree8.dot -o tree8.png

<img src="tree3.png">

<img src="tree8.png">